In [1]:
### Will generate image histogram and stats at 1.25 X to test implementation of feature vector / simpel clusting
import random
import large_image, requests
import numpy as np
import time, glob, os
imagePath = "TCGA-3L-AA1B-01A-01-TS1.9C415218-D5B4-4945-B243-F42A4C8C0484.svs"


fastApiUrl="http://dsapostgisapi:82/"
dsaApiUrl = "https://wsi-deid.pathology.emory.edu/api/v1"

In [2]:
## For now I am using locally storage images to make thngs faster
imageList = os.listdir("SmallSampleFiles/")

print(len(imageList))

14


In [3]:
from collections import deque
 
def split_list(input_list, chunk_size):
  # Create a deque object from the input list
  deque_obj = deque(input_list)
  # While the deque object is not empty
  while deque_obj:
      # Pop chunk_size elements from the left side of the deque object
      # and append them to the chunk list
      chunk = []
      for _ in range(chunk_size):
        if deque_obj:
          chunk.append(deque_obj.popleft())
         
      # Yield the chunk
      yield chunk

In [4]:
def lookupPostgisImageId( imageName, dsaApiUrl):
    r = requests.get(fastApiUrl+"lookupImage", params={"imageName":imageName,"dsaApiUrl":dsaApiUrl})
    #print(r.json())
    return r.json()

def generateRGBGrid(imageName, dsaApiUrl, magnification=1.25, tileSize="native", opType="tileHist"):
    
    ## For now I am using local images, so I need to map the imageName to a localFilePath
    
    imageInfo = lookupPostgisImageId(imageName, dsaApiUrl)
    
    if not imageInfo:
        print("Image is not synced in local database")
        return
        
    imagePath = os.path.join("SmallSampleFiles",imageName)
    
    ## Create a largeImage Pointed
    ts = large_image.open(imagePath)
    computationStats = {"imageName": imageName, "imagePath":imagePath,"opType": opType}
    tilesProcessed = 0
    bytesRead = 0
    
    gridData = []

    ## Compute start time of algorithm
    st = time.time()
    computationStats["startTime"] = st

    if tileSize == "native":
        tileWidth = ts.tileWidth
        tileHeight = ts.tileHeight
    else:
        tileWidth = tileSize
        tileHeight = tileSize

    for tile_info in ts.tileIterator(
        region=dict(units="base_pixels"),
        scale=dict(magnification=magnification),
        tile_size=dict(width=tileWidth, height=tileHeight),
        format=large_image.tilesource.TILE_FORMAT_NUMPY
    ):
        if opType == "tileHist":
            #            im_tile = np.array(tile_info["tile"])
            im_tile = tile_info["tile"]
            bytesRead += im_tile.size
            
            image_array = tile_info['tile']
            avgColor  = np.mean(image_array, axis=(0,  1))

            histData = {'topX':tile_info['gx'],'topY':tile_info['gy'],'width':tile_info['gwidth'],
                        'height':tile_info['gheight'], 'localTileId': tile_info['tile_position']['position'],
                       "apiURL": dsaApiUrl,   "imageId": imageInfo['imageId'],    "imageName": imageInfo['imageName'],
                       "average": list(avgColor), "featureType": "imgHistogram"}

            
            for idx,c in enumerate(['red','green','blue']):
                h = np.histogram(image_array[:,:,idx],bins=255,range=[0,255])
                histData[c] =h[0].tolist()
            gridData.append(histData)
        tilesProcessed += 1
        
        
    ## Export some of the stats for analtsis
    computationStats["elapsedTime"] = time.time() - st
    computationStats["tilesProcessed"] = tilesProcessed
    computationStats["tileWidth"] = tileWidth
    computationStats["tileHeight"] = tileHeight
    computationStats["sizeX"] = ts.sizeX
    computationStats["sizeY"] = ts.sizeY
    computationStats["bytesRead"] = bytesRead
    computationStats["magnification"] = magnification
    computationStats["tileSizeParam"] = tileSize
    computationStats["ftxComputeTime"] = time.time() - st
    computationStats["featureType"] = "imgHistogram"
    computationStats["imageId"] = imageInfo["imageId"]
    

    return computationStats, gridData

In [6]:
url = 'http://dsapostgisapi:82/addTileFeatures'

ftrUrl= 'http://dsapostgisapi:82/addFeatureExtractionParams'

for i in imageList:
    for mag in [10,20]:
        ## Add an endpoint to see if the features have already been computed for this image.
        
        for tsp in ['native']:
        
            imageInfo = lookupPostgisImageId(i, dsaApiUrl)
    #         print(imageInfo)
            paramSet = {'imageId': imageInfo['imageId'],'magnification': mag, 'tileSizeParam': tsp}
            ftrLookupUrl= 'http://dsapostgisapi:82/lookupFeatureExtractionParams'

            r = requests.post(ftrLookupUrl, params=paramSet)
            ##Only want to compute features if we haven't run this set of params yet
#             if not r.json():

            stats,histData = generateRGBGrid(i,dsaApiUrl,magnification=mag,tileSize='native')
            #print(stats)

            r = requests.post(ftrUrl, json=stats)

            ## Delete any ftxtract_id already in the database...
            ftIdParams = {'imageId': imageInfo['imageId'], 'ftxtract_id': r.json()['ftxtract_id']}
            dr = requests.delete("http://dsapostgisapi:82/deleteTileFeatures", params = ftIdParams)
            ## Also add the ftxtract_id that I just pulled into the histograms...
            for h in histData:
                h['paramSetId'] = r.json()['ftxtract_id']
                h['ftxtract_id'] = r.json()['ftxtract_id']

            for l in split_list(histData,1000):
                r = requests.post(url,json=l)
#             else:
#                 print("Already computed features at ",mag,i)

## Add logic to determine is all features/ rois that should be tehre, are there... 


In [ ]:
# import requests


# url = 'http://dsapostgisapi:82/addTileFeatures'


# r = requests.post(url,json=histData)

In [ ]:
histData